## Superposition of one image on another
Tejas Pant <br>
5th September 2018

## Problem 1(a)

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os

os.listdir('../PicsHw2_org/')
images = glob.glob('../PicsHw2_org/*.jpg')

selectRangeImage = 3 #1 = 1a, 2 = 1b 3 = 1c
image = mpimg.imread(images[selectRangeImage-1])
image_org = np.copy(image)
image_jackie = mpimg.imread(images[3])

def getXXprimeFromPQRS(option):
    if option == 1:
        P = [1500, 112]
        Q = [2962, 730]
        R = [1485, 2243]
        S = [3015, 2054]
    elif option == 2:
        P = [1323, 321]
        Q = [3015, 598]
        R = [1299, 2007]
        S = [3033, 1898]
    elif option == 3:
        P = [916, 725]
        Q = [2809, 371]
        R = [890, 2080]
        S = [2847, 2231] 
        
    #Jackie Image (Domain)    
    P_ = [0,0]
    Q_ = [1279, 0]
    R_ = [0, 719]
    S_ = [1279, 719]
    
    #Vectors for caclulation of homography
    #Domain = Jackie, Range = 1abc
    #X_prime = P + Q + R + S #Range points
    #X = P_ + Q_ + R_ + S_ #Domain points
    
    #Domain = 1abc, Range = Jackie
    X = P + Q + R + S 
    X_prime = P_ + Q_ + R_ + S_ 
    
    #Coordinates of frame corners in table format
    XYTablePQRS = np.array([P,R,S,Q],dtype='int32')
    return X_prime, X, XYTablePQRS
    
def calcHomography(X_prime,X):
#Caluclate the Homography H

    A = np.zeros((8,8))
    H = np.ones((3,3))

    for i in range(4):
        H_rowx = [0 for i in range(8)]
        H_rowy = [0 for i in range(8)]

        H_rowx[0] = X[2*i]
        H_rowx[1] = X[2*i+1]
        H_rowx[2] = 1
        H_rowx[6] = -X[2*i]*X_prime[2*i]
        H_rowx[7] = -X[2*i+1]*X_prime[2*i]

        H_rowy[3] = X[2*i]
        H_rowy[4] = X[2*i+1]
        H_rowy[5] = 1
        H_rowy[6] = -X[2*i]*X_prime[2*i+1]
        H_rowy[7] = -X[2*i+1]*X_prime[2*i+1]

        A[2*i] = H_rowx
        A[2*i+1] = H_rowy

    param = np.matmul(np.linalg.inv(A),np.array(X_prime))
    param = np.append(param,1)
    for i in range(3):
        H[i] = param[3*i:3*i+3]
    return H

X_prime_, X_, XYTablePQRS = getXXprimeFromPQRS(selectRangeImage)
H_ = calcHomography(X_prime_,X_)

image_empty = np.zeros(image.shape,dtype='uint8')
image_filled = cv2.fillPoly(image_empty,[XYTablePQRS],(255,255,255))

for i in range(0,image.shape[1]-1): #X-cordinate, col
    for j in range(0,image.shape[0]-1): #Y-coordinate, row
        if any(image_filled[j][i]) > 0:
            X_domain = [i,j]
            X_domain = np.append(np.array(X_domain),1)
            X_range = np.matmul(H_, X_domain)
            X_range = np.rint(X_range/X_range[-1]) #Normalize and round off to nearest integer
            X_range = X_range.astype(int)
            if(X_range[0] < image_jackie.shape[1] and \
               X_range[1] < image_jackie.shape[0]):
                image[j][i] = image_jackie[X_range[1]][X_range[0]]
                

In [ ]:
plt.figure(figsize=(30,30))
plt.subplot(211)
plt.imshow(image_org)
plt.subplot(212)
plt.imshow(image)

## Problem 1(b)

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os

os.listdir('../PicsHw2_org/')
images = glob.glob('../PicsHw2_org/*.jpg')
image1a = mpimg.imread(images[0])
image1c = mpimg.imread(images[2])

def calcHomography(X_prime,X):
#Caluclate the Homography H

    A = np.zeros((8,8))
    H = np.ones((3,3))

    for i in range(4):
        H_rowx = [0 for i in range(8)]
        H_rowy = [0 for i in range(8)]

        H_rowx[0] = X[2*i]
        H_rowx[1] = X[2*i+1]
        H_rowx[2] = 1
        H_rowx[6] = -X[2*i]*X_prime[2*i]
        H_rowx[7] = -X[2*i+1]*X_prime[2*i]

        H_rowy[3] = X[2*i]
        H_rowy[4] = X[2*i+1]
        H_rowy[5] = 1
        H_rowy[6] = -X[2*i]*X_prime[2*i+1]
        H_rowy[7] = -X[2*i+1]*X_prime[2*i+1]

        A[2*i] = H_rowx
        A[2*i+1] = H_rowy

    param = np.matmul(np.linalg.inv(A),np.array(X_prime))
    param = np.append(param,1)
    for i in range(3):
        H[i] = param[3*i:3*i+3]
    return H

In [2]:
#Homography between 1a and 1b
Pa = [1500, 112]
Qa = [2962, 730]
Ra = [1485, 2243]
Sa = [3015, 2054]

Pb = [1323, 321]
Qb = [3015, 598]
Rb = [1299, 2007]
Sb = [3033, 1898]

Pc = [916, 725]
Qc = [2809, 371]
Rc = [890, 2080]
Sc = [2847, 2231] 

#Homography between 1a and 1b 
X_prime_ = Pa + Qa + Ra + Sa 
X_ = Pb + Qb + Rb + Sb
Hab = calcHomography(X_prime_,X_)

#Homography between 1b and 1c
X_prime_ = Pb + Qb + Rb + Sb
X_ = Pc + Qc + Rc + Sc
Hbc = calcHomography(X_prime_,X_)
Hac = np.matmul(Hab,Hbc)

In [ ]:
image_1c_generated = np.zeros(image1c.shape,dtype='uint8')
for i in range(0,image1c.shape[1]-1): #X-cordinate, col
    for j in range(0,image1c.shape[0]-1): #Y-coordinate, row
        X_domain = [i,j]
        X_domain = np.append(np.array(X_domain),1)
        X_range = np.matmul(Hac, X_domain)
        X_range = np.rint(X_range/X_range[-1]) #Normalize and round off to nearest integer
        X_range = X_range.astype(int)
        if(X_range[0] > 0 and X_range[1] > 0 and \
           X_range[0] < image1a.shape[1] and \
           X_range[1] < image1a.shape[0]):
            image_1c_generated[j][i] = image1a[X_range[1]][X_range[0]]

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(121)
plt.title('Fig. 1(a)',fontsize=18)
plt.imshow(image1a)
plt.subplot(122)
plt.title('Fig. 1(c)',fontsize=18)
plt.imshow(image1c)
plt.figure(figsize=(15,15))
plt.title('Projection of Fig. 1(a) on plane of Fig. 1(c)',fontsize=18)
plt.imshow(image_1c_generated)

## Problem 1c

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os

images = glob.glob('../Pics_mine/*.jpg')

selectRangeImage = 3 #1 = 1a, 2 = 1b 3 = 1c
image = mpimg.imread(images[selectRangeImage])
image_org = np.copy(image)
image_jones = mpimg.imread(images[0])

def getXXprimeFromPQRS(option):
    if option == 1:
        P = [1871, 1053]
        Q = [2490, 1019]
        R = [1881, 2275]
        S = [2463, 2046]
    elif option == 2:
        P = [1817, 1103]
        Q = [2699, 1093]
        R = [1851, 2157]
        S = [2685, 2187]
    elif option == 3:
        P = [1904, 1140]
        Q = [2524, 1171]
        R = [1921, 2066]
        S = [2510, 2255] 
        
    #Jones Image (Domain)    
    P_ = [0,0]
    Q_ = [399, 0]
    R_ = [0, 399]
    S_ = [399, 399]
      
    #Domain = 1abc, Range = Jones
    X = P + Q + R + S 
    X_prime = P_ + Q_ + R_ + S_ 
    
    #Coordinates of frame corners in table format
    XYTablePQRS = np.array([P,R,S,Q],dtype='int32')
    return X_prime, X, XYTablePQRS
    
def calcHomography(X_prime,X):
#Caluclate the Homography H

    A = np.zeros((8,8))
    H = np.ones((3,3))

    for i in range(4):
        H_rowx = [0 for i in range(8)]
        H_rowy = [0 for i in range(8)]

        H_rowx[0] = X[2*i]
        H_rowx[1] = X[2*i+1]
        H_rowx[2] = 1
        H_rowx[6] = -X[2*i]*X_prime[2*i]
        H_rowx[7] = -X[2*i+1]*X_prime[2*i]

        H_rowy[3] = X[2*i]
        H_rowy[4] = X[2*i+1]
        H_rowy[5] = 1
        H_rowy[6] = -X[2*i]*X_prime[2*i+1]
        H_rowy[7] = -X[2*i+1]*X_prime[2*i+1]

        A[2*i] = H_rowx
        A[2*i+1] = H_rowy

    param = np.matmul(np.linalg.inv(A),np.array(X_prime))
    param = np.append(param,1)
    for i in range(3):
        H[i] = param[3*i:3*i+3]
    return H

X_prime_, X_, XYTablePQRS = getXXprimeFromPQRS(selectRangeImage)
H_ = calcHomography(X_prime_,X_)

image_empty = np.zeros(image.shape,dtype='uint8')
image_filled = cv2.fillPoly(image_empty,[XYTablePQRS],(255,255,255))

for i in range(0,image.shape[1]-1): #X-cordinate, col
    for j in range(0,image.shape[0]-1): #Y-coordinate, row
        if any(image_filled[j][i]) > 0:
            X_domain = [i,j]
            X_domain = np.append(np.array(X_domain),1)
            X_range = np.matmul(H_, X_domain)
            X_range = np.rint(X_range/X_range[-1]) #Normalize and round off to nearest integer
            X_range = X_range.astype(int)
            if(X_range[0] < image_jones.shape[1] and \
               X_range[1] < image_jones.shape[0]):
                image[j][i] = image_jones[X_range[1]][X_range[0]]
                

In [ ]:
plt.figure(figsize=(30,30))
plt.subplot(211)
plt.imshow(image_org)
plt.subplot(212)
plt.imshow(image)

## Problem 1c , Solution to 1b Task

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os

images = glob.glob('../Pics_mine/*.jpg')
image1a = mpimg.imread(images[1])
image1c = mpimg.imread(images[3])

def calcHomography(X_prime,X):
#Caluclate the Homography H

    A = np.zeros((8,8))
    H = np.ones((3,3))

    for i in range(4):
        H_rowx = [0 for i in range(8)]
        H_rowy = [0 for i in range(8)]

        H_rowx[0] = X[2*i]
        H_rowx[1] = X[2*i+1]
        H_rowx[2] = 1
        H_rowx[6] = -X[2*i]*X_prime[2*i]
        H_rowx[7] = -X[2*i+1]*X_prime[2*i]

        H_rowy[3] = X[2*i]
        H_rowy[4] = X[2*i+1]
        H_rowy[5] = 1
        H_rowy[6] = -X[2*i]*X_prime[2*i+1]
        H_rowy[7] = -X[2*i+1]*X_prime[2*i+1]

        A[2*i] = H_rowx
        A[2*i+1] = H_rowy

    param = np.matmul(np.linalg.inv(A),np.array(X_prime))
    param = np.append(param,1)
    for i in range(3):
        H[i] = param[3*i:3*i+3]
    return H

In [2]:
#Homography between 1a and 1b
Pa = [1871, 1053]
Qa = [2490, 1019]
Ra = [1881, 2275]
Sa = [2463, 2046]

Pb = [1817, 1103]
Qb = [2699, 1093]
Rb = [1851, 2157]
Sb = [2685, 2187]

Pc = [1904, 1140]
Qc = [2524, 1171]
Rc = [1921, 2066]
Sc = [2510, 2255] 

#Homography between 1a and 1b 
X_prime_ = Pa + Qa + Ra + Sa 
X_ = Pb + Qb + Rb + Sb
Hab = calcHomography(X_prime_,X_)

#Homography between 1b and 1c
X_prime_ = Pb + Qb + Rb + Sb
X_ = Pc + Qc + Rc + Sc
Hbc = calcHomography(X_prime_,X_)
Hac = np.matmul(Hab,Hbc)

In [3]:
image_1c_generated = np.zeros(image1c.shape,dtype='uint8')
for i in range(0,image1c.shape[1]-1): #X-cordinate, col
    for j in range(0,image1c.shape[0]-1): #Y-coordinate, row
        X_domain = [i,j]
        X_domain = np.append(np.array(X_domain),1)
        X_range = np.matmul(Hac, X_domain)
        X_range = np.rint(X_range/X_range[-1]) #Normalize and round off to nearest integer
        X_range = X_range.astype(int)
        if(X_range[0] > 0 and X_range[1] > 0 and \
           X_range[0] < image1a.shape[1] and \
           X_range[1] < image1a.shape[0]):
            image_1c_generated[j][i] = image1a[X_range[1]][X_range[0]]

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(121)
plt.title('My Image Similar to Fig. 1(a)',fontsize=18)
plt.imshow(image1a)
plt.subplot(122)
plt.title('My Image Similar to Fig. 1(c)',fontsize=18)
plt.imshow(image1c)
plt.figure(figsize=(15,15))
plt.title('Projection',fontsize=18)
plt.imshow(image_1c_generated)